# SOAP Client - Python Function to use CNPq SOAP plataform

## Needed Instalations

In [166]:
# pip install lxml
# pip install zeep

## Importações

In [167]:
from io import BytesIO, StringIO
from zipfile import ZipFile
import zeep
import xmltodict, json

import io, sys, csv, html, os


## Connection to CNPq SOAP

class Lattes:
    
    def __init__(self, 
                 id_lattes = '7281587998425548', 
                 wsdl = 'http://servicosweb.cnpq.br/srvcurriculo/WSCurriculo?wsdl', 
                 path_to_save_file = 'd:\\Lattes\\'):

        settings = zeep.Settings(strict=True, xml_huge_tree=True) 
        self.lattes = None
        self.lattes_xml = None
        self.lattes_json = None
        self.id = id_lattes
        self.wsdl = wsdl
        self.path = path_to_save_file
    
    def get_zip_lattes(self, id=None):
        if not id == None:
            self.id = id
        elif not hasattr(self, 'id') and not self.id =='':
            id = self.id
        else:
            raise ValueError('You must provide an ID')                
         
        client = zeep.Client(wsdl=self.wsdl)
        self.zip_lattes = client.service.getCurriculoCompactado(self.id)
        self.ocorrencia = client.service.getOcorrenciaCV(self.id)
        return (self.ocorrencia)

    def get_dt_atualizacao (self, id=None):
        if not id == None:
            self.id = id
        elif not hasattr(self, 'id') and not self.id =='':
            id = self.id
            self.get_zip_lattes()
        else:
            raise ValueError('You must provide an ID')                
          
        client = zeep.Client(wsdl=self.wsdl)
        self.dt_atualizacao = client.service.getDataAtualizacaoCV(self.id)
        if self.dt_atualizacao == None:
            raise ValueError('Invalid ID')    
            
    def get_id (self, cpf = "69045542153", nomeCompleto="Alberto de Campos e Silva", dataNascimento="10/05/1976"):
        client = zeep.Client(wsdl=self.wsdl)
        self.id = client.service.getIdentificadorCNPq(cpf, nomeCompleto, dataNascimento)
        if self.id == None:
            raise ValueError('Invalid CPF, Name or DateBirth informed.')
            
    def get_xml_lattes(self, id=None):
        if not id == None:
            self.id = id
        elif not hasattr(self, 'id') and not self.id =='':
            id = self.id
            self.get_zip_lattes()
        else:
            raise ValueError('You must provide an ID')    
        
        if not hasattr(self, 'zip_lattes'):
            self.get_zip_lattes()
        with ZipFile(io.BytesIO(self.zip_lattes)) as myzip:
            with myzip.open(myzip.namelist()[0]) as myfile:
                self.lattes_xml = myfile.read()
        self.json_lattes = xmltodict.parse(self.xml_lattes)
        self.xml_lattes = self.xml_lattes.decode('iso-8859-1').replace("&amp;#10;","\n")
              
    def save_zip_to_disk (self):
        filename = "Lattes_"+ self.id + ".zip"
        file_path = os.path.join(self.path, filename)
        with open(file_path , 'wb') as f:
            f.write(self.zip_lattes)
            
    def save_xml_to_disk (self):
        filename = "Lattes_"+ self.id + ".xml"
        file_path = os.path.join(self.path, filename)
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(self.lattes_xml)
            
    def save_json_to_disk (self):
        filename = "Lattes_"+ self.id + ".json"
        file_path = os.path.join(self.path, filename)
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(self.lattes_json, f)

            

# Unzip Lattes

In [203]:
lattes = Lattes()


In [204]:
print(lattes.get_zip_lattes(id='7281587998425548'))

Curriculo recuperado com sucesso!


In [191]:
lattes.get_id()

In [192]:
print (lattes.id)

7281587998425548


In [173]:
print (lattes.lattes_xml.replace("&amp;#10;","\n"))

<?xml version="1.0" encoding="ISO-8859-1" standalone="no"?><CURRICULO-VITAE SISTEMA-ORIGEM-XML="LATTES_OFFLINE" NUMERO-IDENTIFICADOR="7281587998425548" DATA-ATUALIZACAO="17072021" HORA-ATUALIZACAO="071321"><DADOS-GERAIS NOME-COMPLETO="Alberto de Campos e Silva" NOME-EM-CITACOES-BIBLIOGRAFICAS="Campos Silva, A" NACIONALIDADE="B" PAIS-DE-NASCIMENTO="Brasil" UF-NASCIMENTO="GO" CIDADE-NASCIMENTO="Goiânia" PERMISSAO-DE-DIVULGACAO="NAO" DATA-FALECIMENTO="" SIGLA-PAIS-NACIONALIDADE="BRA" PAIS-DE-NACIONALIDADE="Brasil" ORCID-ID="https://orcid.org/0000-0001-6506-1988"><RESUMO-CV TEXTO-RESUMO-CV-RH="Sou casado com uma mulher maravilhosa e tenho três lindos. Atualmente, sou Analista de C&amp;T no CNPq, lotado na COCEX/CGCEX/DEHS, analisando principalmente processos relacionados à Física. Tenho 45 anos, sou formado em física pela UnB, tendo dado aulas (de física) para o ensino fundamental por 2 anos (concursado GDF). Me mudei para Goiânia (por causa do baixo salário). Lá, trabalhei na justiça por 

### Save to disk

In [174]:
lattes.save_zip_to_disk()

In [175]:
lattes.save_xml_to_disk()

In [176]:
lattes.save_json_to_disk()